In [1]:
from find_cars import find_car, add_heat, apply_threshold, draw_labeled_bboxes
from helper import convert_color

In [2]:
import numpy as np
from collections import deque
from scipy.ndimage.measurements import label
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
# Read and display the image
import matplotlib.image as mpimg
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import pickle

In [3]:
# load SVM model
with open('svc.pkl','rb') as f:
    svc = pickle.load(f)

# load scaler
with open('X_scaler.pkl','rb') as f:
    X_scaler = pickle.load(f)

In [4]:
spatial_size = (32,32)
hist_bins = 32
orient = 9
pix_per_cell = 8
cell_per_block = 2

def process_image(image,single=False):
    """1.Takes in color image, extract features and make prediction
       2.Filter False positive and add back missing over several image frames"""
    
    heat = np.zeros_like(image[:,:,0])
    for scale in [0.7,1,1.3]:
        if scale == 0.7:
            ystart, ystop = 360,600
            xstart, xstop = 0,1280

        elif scale == 1:
            ystart, ystop = 360,600
            xstart, xstop = 0, 1280

        elif scale == 1.3:
            ystart, ystop = 360,720
            xstart, xstop = 0, 1280

        bbox = find_car(image, ystart, ystop, xstart, xstop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
        heat = add_heat(heat, bbox)
    if single:
        heatmap = apply_threshold(heat, 2)
    else:
        buffer_heat.append(heat)
        heat = np.sum(np.array(buffer_heat), axis=0).astype(np.float)
        heat = cv2.GaussianBlur(heat,(15,15),0)
        heatmap = apply_threshold(heat, 10)
    
    labels = label(heatmap) # labels[0] is an array the size of heatmap, labels[1] is the number of labels found
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    
    return draw_img

In [ ]:
bboxes_frame = deque(maxlen=20)
white_output = 'short.mp4'
clip1 = VideoFileClip("/Users/jinjunjie/a_self_driving_car/Advanced line finding/Proj-Advanced-Lane-Lines/project_short.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))